In [5]:
import gradio as gr
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [6]:
# Model ve ön işleme dosyalarının adları
model_filename = 'xgboost_churn_model.joblib'
feature_names_filename = 'model_features.joblib'
scaler_filename = 'scaler.joblib'
label_encoders_filename = 'label_encoders.joblib'
nominal_categories_filename = 'nominal_categories.joblib' # Yeni eklenen

try:
    loaded_model = joblib.load(model_filename)
    loaded_features = joblib.load(feature_names_filename)
    loaded_scaler = joblib.load(scaler_filename)
    loaded_label_encoders = joblib.load(label_encoders_filename)
    loaded_nominal_categories = joblib.load(nominal_categories_filename) # Nominal kategorileri yükle
    print("Tüm model ve ön işleme objeleri başarıyla yüklendi.")
except FileNotFoundError as e:
    print(f"Hata: Gerekli dosyalardan biri bulunamadı: {e}. Lütfen aynı dizinde olduklarından emin olun.")
    exit() # Hata durumunda script'ten çıkmak için

print(f"Yüklü Model: {loaded_model}")
print(f"Yüklü Özellikler (ilk 5): {loaded_features[:5]} ...")
print(f"Yüklü Scaler: {loaded_scaler}")
print(f"Yüklü Label Encoder Anahtarları: {list(loaded_label_encoders.keys())}")
print(f"Yüklü Nominal Kategoriler Anahtarları: {list(loaded_nominal_categories.keys())}")

Tüm model ve ön işleme objeleri başarıyla yüklendi.
Yüklü Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)
Yüklü Özellikler (ilk 5): ['Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married'] ...
Yüklü Scaler: StandardScaler()
Yüklü Label Encoder Anahtarları: ['Gende

#### Kategorik Veriler
##### Label Encoding Uygulanan Veriler
binary_ordinal_cols = [
    'Gender', 'Under 30', 'Senior Citizen', 'Married', 'Dependents',
    'Referred a Friend', 'Phone Service', 'Multiple Lines', 'Internet Service',
    'Online Security', 'Online Backup', 'Device Protection Plan',
    'Premium Tech Support', 'Streaming TV', 'Streaming Music',
    'Streaming Movies', 'Unlimited Data', 'Paperless Billing', 'Churn Label'
]

##### OHE Uygulanan Veriler
nominal_cols = ['City', 'Offer', 'Internet Type', 'Contract', 'Payment Method']

#### Sayısal Veriler
##### Özellik mühendisliği uygulanan veriler
zero_inflated_cols = ['Total Refunds', 'Total Extra Data Charges']

##### Logaritmik dönüşüm uygulanan veriler
skewed_numerical_cols = ['Population', 'Avg Monthly GB Download', 'Total Long Distance Charges', 'Total Revenue']

#### Özellik Ölçeklendirme (Feature Scaling) Uygulanan Veriler
numerical_cols_for_scaling = [
    'Age',
    'Population', 
    'Tenure in Months',
    'Avg Monthly Long Distance Charges',
    'Avg Monthly GB Download', 
    'Monthly Charge',
    'Total Long Distance Charges', 
    'Total Revenue', 
    'Satisfaction Score'
]

In [13]:
# Tahmin Fonksiyonu
def predict_churn(
    gender, under_30, senior_citizen, married, dependents, referred_a_friend,
    phone_service, multiple_lines, internet_service, online_security, online_backup,
    device_protection_plan, premium_tech_support, streaming_tv, streaming_music,
    streaming_movies, unlimited_data, paperless_billing, # Label Encoded (Binary/Ordinal)

    city, offer, internet_type, contract, payment_method, # One-Hot Encoded (Nominal)

    age, population, tenure_in_months, avg_monthly_long_distance_charges,
    avg_monthly_gb_download, monthly_charge, total_long_distance_charges,
    total_revenue, satisfaction_score, # Scaled Numerical

    total_refunds, total_extra_data_charges # Zero Inflated (Sayısal)
):
    # Kullanıcı girdilerini bir sözlüğe dönüştür
    input_data = {
        'Gender': gender,
        'Under 30': under_30,
        'Senior Citizen': senior_citizen,
        'Married': married,
        'Dependents': dependents,
        'Referred a Friend': referred_a_friend,
        'Phone Service': phone_service,
        'Multiple Lines': multiple_lines,
        'Internet Service': internet_service,
        'Online Security': online_security,
        'Online Backup': online_backup,
        'Device Protection Plan': device_protection_plan,
        'Premium Tech Support': premium_tech_support,
        'Streaming TV': streaming_tv,
        'Streaming Music': streaming_music,
        'Streaming Movies': streaming_movies,
        'Unlimited Data': unlimited_data,
        'Paperless Billing': paperless_billing,

        'City': city,
        'Offer': offer,
        'Internet Type': internet_type,
        'Contract': contract,
        'Payment Method': payment_method,

        'Age': age,
        'Population': population,
        'Tenure in Months': tenure_in_months,
        'Avg Monthly Long Distance Charges': avg_monthly_long_distance_charges,
        'Avg Monthly GB Download': avg_monthly_gb_download,
        'Monthly Charge': monthly_charge,
        'Total Long Distance Charges': total_long_distance_charges,
        'Total Revenue': total_revenue,
        'Satisfaction Score': satisfaction_score,

        'Total Refunds': total_refunds,
        'Total Extra Data Charges': total_extra_data_charges
    }

    # Tek bir satırlık DataFrame oluştur
    df_input = pd.DataFrame([input_data])

    # --- Veri Ön İşleme Adımları (Eğitimde uygulanan sırayla!) ---

    # 1. Label Encoding Uygulanan Veriler (Binary/Ordinal Kolonlar)
    binary_ordinal_cols = [
        'Gender', 'Under 30', 'Senior Citizen', 'Married', 'Dependents',
        'Referred a Friend', 'Phone Service', 'Multiple Lines', 'Internet Service',
        'Online Security', 'Online Backup', 'Device Protection Plan',
        'Premium Tech Support', 'Streaming TV', 'Streaming Music',
        'Streaming Movies', 'Unlimited Data', 'Paperless Billing'
    ]

    # Senior Citizen'ın Gradio'dan boolean gelme ihtimaline karşı int'e çevir
    df_input['Senior Citizen'] = df_input['Senior Citizen'].astype(int)

    for col in binary_ordinal_cols:
        if col in loaded_label_encoders:
            le = loaded_label_encoders[col]
            # LabelEncoder'ın görmediği bir değer gelirse hata vermesini engelle
            # ve bu değeri uygun bir şekilde ele al (örn. 0)
            if df_input[col].iloc[0] not in le.classes_:
                # Eğer yeni gelen değer encoder'ın sınıflarında yoksa, varsayılan bir değer kullan
                # veya en yakınını bulmaya çalışın. En basiti 0 atamak olabilir.
                # Daha iyi bir yaklaşım, bilinen sınıflardan birine düşürmektir.
                # Bu örnekte, sadece hata mesajı verip 0 atayacağız.
                print(f"Uyarı: '{col}' sütunu için bilinmeyen değer '{df_input[col].iloc[0]}'. Varsayılan 0 kullanılıyor.")
                df_input[col] = 0
            else:
                df_input[col] = le.transform(df_input[col])
        else:
            # Eğer bir sütun binary_ordinal_cols içinde ama LabelEncoder'ı kaydedilmediyse
            # manuel olarak 'Yes'/'No' gibi değerleri 1/0'a dönüştür.
            # Örneğin, eğer 'Multiple Lines' için 'No phone service' 0'a manuel olarak dönüştürüldüyse.
            if df_input[col].dtype == 'object':
                if df_input[col].iloc[0] == 'Yes':
                    df_input[col] = 1
                elif df_input[col].iloc[0] == 'No':
                    df_input[col] = 0
                elif col == 'Multiple Lines' and df_input[col].iloc[0] == 'No phone service':
                    df_input[col] = 0
                elif col in ['Online Security', 'Online Backup', 'Device Protection Plan',
                             'Premium Tech Support', 'Streaming TV', 'Streaming Music',
                             'Streaming Movies'] and df_input[col].iloc[0] == 'No internet service':
                    df_input[col] = 0


    # 2. One-Hot Encoding Uygulanan Veriler (Nominal Kolonlar)
    nominal_cols = ['City', 'Offer', 'Internet Type', 'Contract', 'Payment Method']
    # ÖNEMLİ: drop_first=True olarak ayarlandı!
    df_input_ohe = pd.get_dummies(df_input, columns=nominal_cols, drop_first=True)

    # 3. Logaritmik Dönüşüm Uygulama
    skewed_numerical_cols = ['Population', 'Avg Monthly GB Download', 'Total Long Distance Charges', 'Total Revenue']
    for col in skewed_numerical_cols:
        if col in df_input_ohe.columns:
            # np.log1p(x) = log(1+x), sıfırları da işler.
            df_input_ohe[col] = np.log1p(df_input_ohe[col])

    # 4. Özellik Ölçeklendirme (Feature Scaling) Uygulanan Veriler
    numerical_cols_for_scaling = [
        'Age', 'Population', 'Tenure in Months', 'Avg Monthly Long Distance Charges',
        'Avg Monthly GB Download', 'Monthly Charge', 'Total Long Distance Charges',
        'Total Revenue', 'Satisfaction Score'
    ]

    cols_to_scale = [col for col in numerical_cols_for_scaling if col in df_input_ohe.columns]
    if cols_to_scale:
        df_input_ohe[cols_to_scale] = loaded_scaler.transform(df_input_ohe[cols_to_scale])


    # --- Kolonların Modelin Beklediği Sıraya Getirilmesi (Çok Önemli!) ---
    # loaded_features, eğitimdeki tüm kolonların doğru sırasını içeriyor olmalı (drop_first=True ile oluşmuş haliyle).
    # df_input_ohe DataFrame'ini, loaded_features listesindeki sütunlarla yeniden indeksle.
    # Olmayan sütunlar 0 ile doldurulacak, fazla sütunlar atılacak.
    df_final = df_input_ohe.reindex(columns=loaded_features, fill_value=0)


    # Model ile tahmin yap
    prediction_proba = loaded_model.predict_proba(df_final)[0][1] # Churn olasılığı (1 sınıfının olasılığı)
    prediction_class = loaded_model.predict(df_final)[0] # 0 veya 1

    # Sonucu metin olarak formatla
    churn_status = "Customer will churn" if prediction_class == 1 else "Customer won't churn"
    result_text = f"{churn_status} (Churn Prediction: %{prediction_proba*100:.2f})"

    return result_text

In [14]:
# Gradio giriş bileşenleri
inputs = [
    # Label Encoding Uygulananlar
    gr.Radio(["Male", "Female"], label="Gender"),
    gr.Radio(["Yes", "No"], label="Under 30"),
    gr.Checkbox(label="Senior Citizen (Yes/No)"),
    gr.Radio(["Yes", "No"], label="Married"),
    gr.Radio(["Yes", "No"], label="Dependents"),
    gr.Radio(["Yes", "No"], label="Referred a Friend"),
    gr.Radio(["Yes", "No"], label="Phone Service"),
    gr.Radio(["Yes", "No", "No phone service"], label="Multiple Lines"),
    # Internet Service - eğer binary_ordinal_cols içinde Yes/No ise aşağıdaki gibi kalsın.
    # Eğer bu kolon aslında 3 veya daha fazla kategoriye sahip ve nominal_cols içinde OHE olduysa,
    # o zaman bu satırı silip Internet Type'ı kullanın (aşağıda).
    gr.Radio(["Yes", "No"], label="Internet Service"),
    gr.Radio(["Yes", "No", "No internet service"], label="Online Security"),
    gr.Radio(["Yes", "No", "No internet service"], label="Online Backup"),
    gr.Radio(["Yes", "No", "No internet service"], label="Device Protection Plan"),
    gr.Radio(["Yes", "No", "No internet service"], label="Premium Tech Support"),
    gr.Radio(["Yes", "No", "No internet service"], label="Streaming TV"),
    gr.Radio(["Yes", "No", "No internet service"], label="Streaming Music"),
    gr.Radio(["Yes", "No", "No internet service"], label="Streaming Movies"),
    gr.Radio(["Yes", "No"], label="Unlimited Data"),
    gr.Radio(["Yes", "No"], label="Paperless Billing"),

    # OHE Uygulananlar (Nominal) - Yüklü kategorilere göre dinamik oluşturuluyor
    gr.Dropdown(loaded_nominal_categories.get('City', []), label="City"),
    gr.Dropdown(loaded_nominal_categories.get('Offer', []), label="Offer"),
    gr.Dropdown(loaded_nominal_categories.get('Internet Type', []), label="Internet Type"),
    gr.Dropdown(loaded_nominal_categories.get('Contract', []), label="Contract"),
    gr.Dropdown(loaded_nominal_categories.get('Payment Method', []), label="Payment Method"),

    # Sayısal Veriler (Ölçeklenmiş)
    gr.Slider(minimum=18, maximum=80, step=1, label="Age"),
    gr.Slider(minimum=0, maximum=100000, step=1, label="Population"),
    gr.Slider(minimum=0, maximum=72, step=1, label="Tenure in Months"),
    gr.Slider(minimum=0.0, maximum=100.0, step=0.01, label="Avg Monthly Long Distance Charges"),
    gr.Slider(minimum=0.0, maximum=1000.0, step=0.01, label="Avg Monthly GB Download"),
    gr.Slider(minimum=0.0, maximum=120.0, step=0.01, label="Monthly Charge"),
    gr.Slider(minimum=0.0, maximum=5000.0, step=0.01, label="Total Long Distance Charges"),
    gr.Slider(minimum=0.0, maximum=10000.0, step=0.01, label="Total Revenue"),
    gr.Slider(minimum=1, maximum=5, step=1, label="Satisfaction Score"),

    # Sayısal Veriler (Zero Inflated)
    gr.Slider(minimum=0.0, maximum=50.0, step=0.01, label="Total Refunds"),
    gr.Slider(minimum=0.0, maximum=50.0, step=0.01, label="Total Extra Data Charges")
]

output = gr.Textbox(label="Churn Tahmini")

In [15]:
# Gradio arayüzünü oluştur
if ('loaded_model' in locals() and 'loaded_features' in locals() and
    'loaded_scaler' in locals() and 'loaded_label_encoders' in locals() and
    'loaded_nominal_categories' in locals()):
    iface = gr.Interface(
        fn=predict_churn,
        inputs=inputs,
        outputs=output,
        title="Müşteri Churn Tahmini Uygulaması",
        description="Müşteri özelliklerini girerek churn etme olasılığını tahmin edin. (Bu demo sadece tahmini bir örnektir, gerçek uygulamalar daha fazla doğrulama gerektirir.)"
    )

    # Arayüzü başlat
    iface.launch()
else:
    print("Gerekli model veya ön işleme objeleri yüklenemediği için Gradio arayüzü başlatılamadı.")

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Uyarı: 'Gender' sütunu için bilinmeyen değer 'Female'. Varsayılan 0 kullanılıyor.
Uyarı: 'Senior Citizen' sütunu için bilinmeyen değer '1'. Varsayılan 0 kullanılıyor.
